## Lab Overview
- In this lab, we will explore PySpark SQL, which is Spark’s high-level API for working with structured data. You will learn how to interact with PySparkSQL, using the DataFrame API and SQL query.
- Before you can use SQL in Spark, you will first need to create a temporary view. Creating a temporary view is performed by calling the createOrReplaceTempView() function on a DataFrame. Once created, pass any SQL statements in a call to spark.sql():
### Learning Objective:
- This lab aims to equip you with the practical skills to leverage PySpark SQL for querying Spark DataFrames. By the end of this lab, you will be able to:
    - Import data from a CSV file into a Spark DataFrame.
    - Register a DataFrame as a temporary view.
    - Execute SQL queries against Spark DataFrames using `spark.sql()`.
    - Perform various SQL operations, including filtering, grouping, sorting, and aggregation.
    - Apply advanced SQL techniques like scalar subqueries and regular expressions within PySpark.
    - Analyze and extract meaningful insights from data using PySpark SQL.


In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
# Initialize SparkSession
spark = SparkSession.builder.appName('Carapp').getOrCreate()

# Read CSV file with Spark DF API to return data in a DF
cardf = spark.read.load("Data/cars.csv", format="csv", header=True, inferSchema=True)
cardf.printSchema()

root
 |-- Car: string (nullable = true)
 |-- MPG: double (nullable = true)
 |-- Cylinders: integer (nullable = true)
 |-- Displacement: double (nullable = true)
 |-- Horsepower: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Acceleration: double (nullable = true)
 |-- Model: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- city: string (nullable = true)



#### Example 1: Return all data from CSV file

In [3]:
# Register DF as a temporary SQL table
cardf.createTempView("dealer") # dealer is used as a table name
spark.sql("SELECT * FROM dealer").show(10)

+--------------------+----+---------+------------+----------+------+------------+-----+------+--------+-------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|quantity|   city|
+--------------------+----+---------+------------+----------+------+------------+-----+------+--------+-------+
|AMC Ambassador Br...|13.0|        8|       360.0|       175|  3821|        11.0|   73|    US|      25|NewYork|
|  AMC Ambassador DPL|15.0|        8|       390.0|       190|  3850|         8.5|   70|    US|       2|     NJ|
|  AMC Ambassador SST|17.0|        8|       304.0|       150|  3672|        11.5|   72|    US|       4| DALLAS|
|         AMC Concord|19.4|        6|       232.0|        90|  3210|        17.2|   78|    US|      52|  TEXAS|
|         AMC Concord|24.3|        4|       151.0|        90|  3003|        20.1|   80|    US|      42|     OH|
|     AMC Concord d/l|18.1|        6|       258.0|       120|  3410|        15.1|   78|    US|       4|N

#### Example 2:  Find the sum of quantity per Dealer. Group by `car.`

In [4]:
spark.sql("SELECT Car, sum(quantity) FROM dealer GROUP BY Car").show()

+--------------------+-------------+
|                 Car|sum(quantity)|
+--------------------+-------------+
|Chevrolete Chevel...|          206|
|Volkswagen 1131 D...|           63|
|Chevrolet Monte C...|          582|
|     Ford LTD Landau|          331|
|       Honda Prelude|          400|
|      Chevrolet Nova|          608|
|   Volkswagen Rabbit|         1441|
|     Ford Torino 500|           81|
|        Toyota Camry|          153|
|         Audi 100 LS|           65|
|Plymouth Valiant ...|          197|
|Toyota Corolla Ma...|          127|
|Oldsmobile Cutlas...|          752|
|Fiat 124 Sport Coupe|          159|
|Chevrolet Caprice...|          732|
|     Volvo 145e (sw)|          121|
|            Audi Fox|          186|
|    Chevrolet Camaro|          438|
|       Dodge Aspen 6|          329|
|AMC Ambassador Br...|           25|
+--------------------+-------------+
only showing top 20 rows



#### Example 3:  Multiple aggregations.
- a. Sum of quantity per dealership.
- b. Max quantity per dealership.

In [5]:
spark.sql("SELECT Car, sum(quantity) AS sum, max(quantity) AS max FROM dealer GROUP BY Car ORDER BY Car").show()

+--------------------+---+---+
|                 Car|sum|max|
+--------------------+---+---+
|AMC Ambassador Br...| 25| 25|
|  AMC Ambassador DPL|  2|  2|
|  AMC Ambassador SST|  4|  4|
|         AMC Concord| 94| 52|
|      AMC Concord DL| 45| 45|
|    AMC Concord DL 6|328|328|
|     AMC Concord d/l|  4|  4|
|         AMC Gremlin|512|214|
|          AMC Hornet|404|172|
|AMC Hornet Sporta...| 90| 90|
|         AMC Matador|833|234|
|    AMC Matador (sw)|302|185|
|           AMC Pacer|221|221|
|       AMC Pacer d/l| 75| 75|
|       AMC Rebel SST|  4|  4|
|  AMC Rebel SST (sw)| 53| 53|
|       AMC Spirit DL|575|575|
|         Audi 100 LS| 65| 65|
|          Audi 100LS|209|164|
|           Audi 4000|362|362|
+--------------------+---+---+
only showing top 20 rows



#### Example 4: Count the number of distinct dealer cities per car.

In [6]:
spark.sql("SELECT Car, count(DISTINCT city) AS count FROM dealer GROUP BY Car").show()

+--------------------+-----+
|                 Car|count|
+--------------------+-----+
|Chevrolete Chevel...|    1|
|Volkswagen 1131 D...|    1|
|Chevrolet Monte C...|    2|
|     Ford LTD Landau|    1|
|       Honda Prelude|    1|
|      Chevrolet Nova|    3|
|   Volkswagen Rabbit|    5|
|     Ford Torino 500|    1|
|        Toyota Camry|    1|
|         Audi 100 LS|    1|
|Plymouth Valiant ...|    1|
|Toyota Corolla Ma...|    1|
|Oldsmobile Cutlas...|    1|
|Fiat 124 Sport Coupe|    1|
|Chevrolet Caprice...|    3|
|     Volvo 145e (sw)|    1|
|    Chevrolet Camaro|    1|
|            Audi Fox|    1|
|       Dodge Aspen 6|    1|
|    Pontiac Catalina|    3|
+--------------------+-----+
only showing top 20 rows



#### Example 5: Find the cars that have quantities greater than 200 and display data in descending order by quantity.

In [7]:
spark.sql("SELECT * FROM dealer WHERE quantity > 200 ORDER BY quantity desc").show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+--------+------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|quantity|  city|
+--------------------+----+---------+------------+----------+------+------------+-----+------+--------+------+
|   Plymouth Arrow GS|25.5|        4|       122.0|        96|  2300|        15.5|   77|    US|   75275|    OH|
|Buick Opel Isuzu ...|30.0|        4|       111.0|        80|  2155|        14.8|   77|    US|     814|    OH|
|      Toyota Mark II|19.0|        6|       156.0|       108|  2930|        15.5|   76| Japan|     757| TEXAS|
|Oldsmobile Cutlas...|17.0|        8|       260.0|       110|  4060|        19.0|   77|    US|     752|    NJ|
|       AMC Spirit DL|27.4|        4|       121.0|        80|  2670|        15.0|   79|    US|     575|    OH|
|     Ford Mustang GL|27.0|        4|       140.0|        86|  2790|        15.6|   82|    US|     439|    OH|
|

#### Example 6: Scalar Subquery in the `WHERE` clause.

- Find the cars that have a horsepower greater than “average.”


In [9]:
spark.sql("SELECT * FROM dealer WHERE Horsepower > (SELECT avg(Horsepower) FROM dealer)").show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+--------+-------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|quantity|   city|
+--------------------+----+---------+------------+----------+------+------------+-----+------+--------+-------+
|AMC Ambassador Br...|13.0|        8|       360.0|       175|  3821|        11.0|   73|    US|      25|NewYork|
|  AMC Ambassador DPL|15.0|        8|       390.0|       190|  3850|         8.5|   70|    US|       2|     NJ|
|  AMC Ambassador SST|17.0|        8|       304.0|       150|  3672|        11.5|   72|    US|       4| DALLAS|
|     AMC Concord d/l|18.1|        6|       258.0|       120|  3410|        15.1|   78|    US|       4|NewYork|
|AMC Hornet Sporta...|18.0|        6|       258.0|       110|  2962|        13.5|   71|    US|      90|     NJ|
|         AMC Matador|14.0|        8|       304.0|       150|  3672|        11.5|   73|    US|     131| 

#### Example 7: Find the sum of only 'Toyota Camrys' and 'Dodge D200s' per dealership.

In [17]:
spark.sql("SELECT Car, sum(quantity) FILTER (WHERE Car IN ('Toyota Camry', 'Honda Prelude')) AS `sum(quantity)` FROM dealer GROUP BY Car").show(20)

+--------------------+-------------+
|                 Car|sum(quantity)|
+--------------------+-------------+
|Chevrolete Chevel...|         NULL|
|Volkswagen 1131 D...|         NULL|
|Chevrolet Monte C...|         NULL|
|     Ford LTD Landau|         NULL|
|       Honda Prelude|          400|
|      Chevrolet Nova|         NULL|
|   Volkswagen Rabbit|         NULL|
|     Ford Torino 500|         NULL|
|        Toyota Camry|          153|
|         Audi 100 LS|         NULL|
|Plymouth Valiant ...|         NULL|
|Toyota Corolla Ma...|         NULL|
|Oldsmobile Cutlas...|         NULL|
|Fiat 124 Sport Coupe|         NULL|
|Chevrolet Caprice...|         NULL|
|     Volvo 145e (sw)|         NULL|
|            Audi Fox|         NULL|
|    Chevrolet Camaro|         NULL|
|       Dodge Aspen 6|         NULL|
|AMC Ambassador Br...|         NULL|
+--------------------+-------------+
only showing top 20 rows



#### Example 8: Regular Expressions

In [18]:
spark.sql("SELECT Car, REGEXP_EXTRACT(Car, ' ([A-Za-z]+)\.') AS CarTitle FROM dealer").show(10)

+--------------------+----------+
|                 Car|  CarTitle|
+--------------------+----------+
|AMC Ambassador Br...|Ambassador|
|  AMC Ambassador DPL|Ambassador|
|  AMC Ambassador SST|Ambassador|
|         AMC Concord|    Concor|
|         AMC Concord|    Concor|
|     AMC Concord d/l|   Concord|
|      AMC Concord DL|   Concord|
|    AMC Concord DL 6|   Concord|
|         AMC Gremlin|    Gremli|
|         AMC Gremlin|    Gremli|
+--------------------+----------+
only showing top 10 rows

